In [2]:
import pandas as pd
from difflib import SequenceMatcher
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm

In [5]:
# 데이터 불러오기
apt_df = pd.read_csv('./전세_아파트_리스트.csv', encoding='cp949')
apart_df = pd.read_csv('./전세_전처리_데이터.csv')
apart_df=apart_df[~apart_df['단지명'].str.contains("임대")]
apart_df.reset_index(drop=True, inplace=True)

In [6]:
apart_df

,주소,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명,동_단지,시군구,번지
0,서울특별시 강남구 개포동 655-2,개포2차현대아파트(220),전세,77.75,201501,13,"40,000",5,1988.0,언주로 103,개포동 개포2차현대아파트(220),서울특별시 강남구 개포동,655-2
1,서울특별시 강남구 개포동 655-2,개포2차현대아파트(220),전세,77.75,201503,2,"50,000",2,1988.0,언주로 103,개포동 개포2차현대아파트(220),서울특별시 강남구 개포동,655-2
2,서울특별시 강남구 개포동 655-2,개포2차현대아파트(220),전세,77.75,201505,23,"50,000",6,1988.0,언주로 103,개포동 개포2차현대아파트(220),서울특별시 강남구 개포동,655-2
3,서울특별시 강남구 개포동 655-2,개포2차현대아파트(220),전세,77.75,201506,8,"49,000",3,1988.0,언주로 103,개포동 개포2차현대아파트(220),서울특별시 강남구 개포동,655-2
4,서울특별시 강남구 개포동 655-2,개포2차현대아파트(220),전세,77.75,201507,22,"50,000",9,1988.0,언주로 103,개포동 개포2차현대아파트(220),서울특별시 강남구 개포동,655-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
818132,서울특별시 중랑구 중화동 450,한신아파트(103~109),전세,50.37,202204,2,"37,000",21,1997.0,동일로 752,중화동 한신아파트(103~109),서울특별시 중랑구 중화동,450
818133,서울특별시 중랑구 중화동 450,한신아파트(103~109),전세,50.37,202204,2,"37,000",21,1997.0,동일로 752,중화동 한신아파트(103~109),서울특별시 중랑구 중화동,450
818134,서울특별시 중랑구 중화동 450,한신아파트(103~109),전세,59.76,202204,16,"46,000",4,1997.0,동일로 752,중화동 한신아파트(103~109),서울특별시 중랑구 중화동,450
818135,서울특별시 중랑구 중화동 450,한신아파트(103~109),전세,84.69,202204,27,"51,000",7,1997.0,동일로 752,중화동 한신아파트(103~109),서울특별시 중랑구 중화동,450


In [7]:
len(apart_df['동_단지'].unique())

8865

In [8]:
# 문제 데이터 갯수 확인
len(apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index)

4159

In [9]:
# 빈 리스트 정의
apt_lst2 = []

In [11]:
apt_df=apt_df[apt_df['road']!=" "]
apt_df

,apt,road,new_apt,new_road,latitude,longitude,company,home_cnt,home_size,room_cnt,bath_cnt,bigo,zibun,new_zibun
0,개포동 개포2차현대아파트(220),언주로 103,개포현대2차(220동),개포동,37.481215,127.052733,현대산업개발(주),54,77.75,3,1,None,None,None
1,개포동 개포6차우성아파트1동~8동,언주로 3,정든우성6단지,불정로 195,37.361885,127.122656,수도배관(공용배관) 교체완료(2014년),706,84.97,3,2,None,None,None
2,개포동 개포우성3차,개포로 307,우성3차,개포로 307,37.483998,127.056011,우성건설(주),405,104.43,3,1,None,None,None
3,개포동 개포우성3차,개포로 307,우성3차,개포로 307,37.483998,127.056011,우성건설(주),405,133.46,4,2,None,None,None
4,개포동 개포우성3차,개포로 307,우성3차,개포로 307,37.483998,127.056011,우성건설(주),405,161.00,5,2,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31043,장안동 에스아이팰리스장안센텀,한천로2길 84,에스아이팰리스장안센텀(주상복합),한천로2길 84,37.562386,127.063230,(주)성우에스아이,99,28.75,2,1,None,None,None
31044,장안동 에스아이팰리스장안센텀,한천로2길 84,에스아이팰리스장안센텀(주상복합),한천로2길 84,37.562386,127.063230,(주)성우에스아이,99,47.20,3,1,None,None,None
31045,장안동 청명에버아트2차,장한로21길 19-15,청명에버아트2차(주상복합),장한로21길 19-15,37.569645,127.068760,주식회사지성건설,18,37.07,1,1,None,None,None
31046,장안동 청명에버아트2차,장한로21길 19-15,청명에버아트2차(주상복합),장한로21길 19-15,37.569645,127.068760,주식회사지성건설,18,48.72,2,1,None,None,None


In [ ]:
apart_df[apart_df['단지명']=='연세7차B동']

In [247]:
a = apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index[3242]
apt = apt_df.apt[a]
road = apt_df.road[a]
apt, road

('창천동 하이펠리스', '연희로2안길 7-22')

In [12]:
# 데이터 수집
for a1 in range(2800, len(apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index) ):
    try:
        try:
            a = apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index[a1]
            apt = apt_df.apt[a]
            road = apt_df.road[a]
            sch = (apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'] + ' ' + apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['번지']).unique()[0]
            sch_road = apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split(' ')[-2] + ' ' + road
            zibun = sch.split('특별시 ')[-1]
            apt_nm = 'None'
            road_nm = 'None'
            lat = '0'
            long = '0'
            home = '0'
            apt_company = 'None'
            size = '0'
            room = '0'
            bath = '0'
            bigo = 'None'
            zibun_nm = 'None'
            rt = 0
            max_i = 0
            web = Chrome('C:/pydata/chromedriver.exe')
            web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch)
            web.implicitly_wait(5)

            try:
                web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
                apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('도로명')[0]
            except:
                web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch_road)
                web.implicitly_wait(5)
                try:
                    web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
                    apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('지번')[0]
                except:
                    apt_nm=''


            if apt_nm != '':
                web.get('https://new.land.naver.com/complexes/667?ms=37.483998,127.056011,17&a=APT:ABYG:JGC&e=RETAIL')
                web.implicitly_wait(5)
                web.find_element(By.ID, 'search_input').clear()
                web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                web.implicitly_wait(5)

                try:
                    web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_fixed > div')
                    try:
                        web.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div[2]/div/div/div/div/i')
                        print('error', a1,  a, apt)
                        apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                    except:
                        num_lst = []
                        for i in range(len(web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div').find_elements(By.CLASS_NAME, 'item'))):
                            if (len(web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a').find_elements(By.CSS_SELECTOR,'*')) > 1):
                                if (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.info_area > strong').text == '아파트') & (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.address').text.endswith(apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split('특별시 ')[-1])):
                                    num_lst.append(i)
                        if len(num_lst)==1:
                            max_i = num_lst[0]
                        elif len(num_lst) > 1:
                            for ii in num_lst:
                                web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(ii+1)+')').click()
                                web.implicitly_wait(5)
                                web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                                web.implicitly_wait(5)
                                if (web.find_elements(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(2)')[-1].text.split('구 ')[-1].split(' ')[0].endswith(road.split(' ')[0])) or (web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('-')[0].endswith(sch.split('특별시 ')[-1].split('-')[0])):
                                    zibun_nm1 = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                                    road_nm1 = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                                    if max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio()) > rt:
                                        rt = max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio())
                                        max_i = ii
                                web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                                web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                                web.implicitly_wait(5)
                        web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(max_i+1)+')').click()
                        web.implicitly_wait(5)
                        web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                        web.implicitly_wait(5)
                        zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                        road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                        apt_nm = web.find_element(By.ID,'complexTitle').text
                        lat = web.current_url.split('=')[1].split(',')[0]
                        long = web.current_url.split('=')[1].split(',')[1]
                        home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                        apt_company = web.find_elements(By.CLASS_NAME, 'info_table_item')[3].find_element(By.CLASS_NAME, 'table_td').text
                        try:
                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                        except:
                            pass
                        for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                            web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                            web.implicitly_wait(3)
                            size = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[1]/td').text.split('㎡')[1][1:]
                            room = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[0][:1]
                            bath = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[1][:1]

                            apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})

                except:
                    try:
                        web.find_element(By.CSS_SELECTOR, '#summaryInfo')
                        web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                        web.implicitly_wait(5)
                        zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                        road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                        apt_nm = web.find_element(By.ID,'complexTitle').text
                        lat = web.current_url.split('=')[1].split(',')[0]
                        long = web.current_url.split('=')[1].split(',')[1]
                        home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                        apt_company = web.find_elements(By.CLASS_NAME, 'info_table_item')[3].find_element(By.CLASS_NAME, 'table_td').text
                        try:
                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                        except:
                            pass
                        for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                            web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                            web.implicitly_wait(3)
                            size = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(1) > td').text.split('㎡')[1][1:]
                            room = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(2) > td').text.split('/')[0][:1]
                            bath = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(2) > td').text.split('/')[1][:1]

                            apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})

                    except:
                            web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch_road)
                            web.implicitly_wait(5)
                            try:
                                web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
                                apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('지번')[0]
                            except:
                                apt_nm = ''
                                bigo='아파트없음'
                                print('error', a1, a, apt)
                                apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})

                            if apt_nm != '':
                                web.get('https://new.land.naver.com/complexes/667?ms=37.483998,127.056011,17&a=APT:ABYG:JGC&e=RETAIL')
                                web.implicitly_wait(5)
                                web.find_element(By.ID, 'search_input').clear()
                                web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                                web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                                web.implicitly_wait(5)
                                try:
                                    web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_fixed > div')
                                    try:
                                        web.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div[2]/div/div/div/div/i')
                                        print('error', a1,  a, apt)
                                        apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                                    except:
                                        num_lst = []
                                        for i in range(len(web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div').find_elements(By.CLASS_NAME, 'item'))):
                                            if (len(web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a').find_elements(By.CSS_SELECTOR,'*')) > 1):
                                                if (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.info_area > strong').text == '아파트') & (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.address').text.endswith(apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split('특별시 ')[-1])):
                                                    num_lst.append(i)
                                        if len(num_lst)==1:
                                            max_i = num_lst[0]
                                        elif len(num_lst)>1:
                                            for i in num_lst:
                                                web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+')').click()
                                                web.implicitly_wait(5)
                                                web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                                                web.implicitly_wait(5)
                                                if (web.find_elements(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(2)')[-1].text.split('구 ')[-1].split(' ')[0].endswith(road.split(' ')[0])) and (web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('-')[0].endswith(sch.split('특별시 ')[-1].split('-')[0])):
                                                    zibun_nm1 = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                                                    road_nm1 = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                                                    if max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio()) > rt:
                                                        rt = max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio())
                                                        max_i = i
                                                web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                                                web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                                                web.implicitly_wait(5)
                                        web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(max_i+1)+')').click()
                                        web.implicitly_wait(5)
                                        web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                                        web.implicitly_wait(5)
                                        zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                                        road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                                        apt_nm = web.find_element(By.ID,'complexTitle').text
                                        lat = web.current_url.split('=')[1].split(',')[0]
                                        long = web.current_url.split('=')[1].split(',')[1]
                                        home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                                        try:
                                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                                            web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                                        except:
                                            pass
                                        for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                                            web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                                            web.implicitly_wait(3)
                                            size = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[1]/td').text.split('㎡')[1][1:]
                                            room = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[0][:1]
                                            bath = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[1][:1]

                                            apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                                except:
                                    print('error', a1, a, apt)
                                    bigo = '아파트없음'
                                    apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
            else:
                print('error', a1, a, apt)
                bigo = '아파트없음'
                apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})

        except:
                print('error', a1, a, apt)
                bigo = '아파트없음'
    except:
        print('name_error', a1, a, apt)

C:\Users\jhgf\AppData\Local\Temp\ipykernel_11924\185743063.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  web = Chrome('C:/pydata/chromedriver.exe')


error 2800 23203 진관동 은평뉴타운상림마을(현대아이파크B-4)
error 2801 23204 진관동 은평뉴타운상림마을(현대아이파크B-4)
error 2802 23205 진관동 은평뉴타운상림마을(현대아이파크B-4)
error 2803 23206 진관동 은평뉴타운우물골(두산위브4단지)
error 2804 23207 진관동 은평뉴타운우물골(두산위브5단지)
error 2805 23208 진관동 은평뉴타운우물골(두산위브6단지)
error 2806 23209 진관동 은평뉴타운우물골(두산위브7단지)
error 2807 23210 진관동 은평뉴타운우물골(두산위브8단지)
error 2808 23215 진관동 은평뉴타운폭포동(힐스테이트D-11)
error 2809 23216 진관동 은평뉴타운폭포동(힐스테이트D-9)
error 2810 23248 내수동 경희궁파크팰리스
error 2811 23259 당주동 롯데미도파광화문빌딩
error 2813 23300 부암동 별빛고운
error 2816 23322 숭인동 숭인동103동(569-4)
error 2817 23323 숭인동 숭인동104동(569-2)
error 2818 23324 숭인동 숭인동105동(569-3)
error 2819 23333 숭인동 숭인한양LEEPS
error 2824 23362 연건동 (60-1)
error 2840 23438 창신동 성용
error 2847 23569 신당동 (432-904)
error 2848 23570 신당동 GV
error 2849 23571 신당동 각산빌
error 2866 23588 신당동 글로리안(B)
error 2867 23635 신당동 도현블랑빌
error 2868 23636 신당동 동산빌라(2차)
error 2869 23637 신당동 동평화패션타운
error 2872 23678 신당동 신신(1차)
error 2873 23679 신당동 약수(372-13)
error 2874 23688 신당동 인성아트빌
error 2875 23715 신당동 탑힐
error 2877 23

error 3556 28159 방학동 우성아파트1
error 3557 28160 방학동 우성아파트1
error 3558 28161 방학동 우성아파트1
error 3559 28162 방학동 우성아파트1
error 3562 28196 장안동 가우디트윈캐슬2
error 3563 28197 장안동 대성하이빌(B동)
error 3566 28202 장안동 젬스톤힐즈
error 3567 28224 노량진동 벧엘ForZoom
error 3571 28243 상도동 백촌빌라트
error 3576 28319 남가좌동 디엠씨엘가
error 3593 28336 연희동 로엠아트빌(A동)
error 3594 28337 연희동 성원쌍떼빌팰리스
error 3595 28338 연희동 연희궁그린빌라트
error 3608 28375 반포동 몽마르뜨파크
error 3611 28416 방배동 TheHerren
error 3613 28456 방배동 서리풀e편한세상
error 3614 28457 방배동 서리풀e편한세상
error 3615 28458 방배동 서리풀e편한세상
error 3616 28459 방배동 서리풀e편한세상
error 3617 28460 방배동 서리풀e편한세상
error 3618 28461 방배동 서리풀e편한세상
error 3619 28462 방배동 서리풀e편한세상
error 3620 28463 방배동 서리풀e편한세상
error 3621 28484 서초동 N-Suite101동(1511-6)
error 3622 28485 서초동 N-Suite102동(1511-3)
error 3623 28489 서초동 다솔멤버스
error 3658 28614 길음동 금호
error 3661 28663 마천동 마천금호어울림1차아파트
error 3662 28664 마천동 마천금호어울림1차아파트
error 3663 28665 마천동 마천금호어울림1차아파트
error 3664 28666 마천동 마천금호어울림1차아파트
error 3665 28667 마천동 마천금호어울림1차아파트
error 3666 28668 마천동

In [14]:
# 데이터프레임 확인
pd.DataFrame(apt_lst2).to_csv("2800-4159.csv", index=False)